In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
filepath='../input/agriculture-crop-images/crop_images/'
sub_class=os.listdir(filepath)
sub_class

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig=plt.figure(figsize=(10,5))
path=os.path.join(filepath,sub_class[1])
path

In [ ]:
filenames=os.listdir(path)
filenames[:3]
j=1
for i in filenames[:4]:
    img=plt.imread(os.path.join(path,i))
    plt.subplot(240+j)
    j=j+1
    plt.imshow(img)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Flatten,MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
path_train1='../input/agriculture-crop-images/crop_images/'
path_train2='../input/agriculture-crop-images/kag2/'
path_train3='../input/agriculture-crop-images/some_more_images/'
path_test='../input/agriculture-crop-images/test_crop_image/'

In [ ]:
Datagen=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,
                          horizontal_flip=True,rescale=1/255)

In [ ]:
train_gen1=Datagen.flow_from_directory(directory=path_train1,
                                      target_size=(224,224),
                                      color_mode='rgb',
                                      batch_size=32,
                                      class_mode='categorical',
                                      subset='training',
                                      shuffle=True)
train_gen2=Datagen.flow_from_directory(directory=path_train2,
                                      target_size=(224,224),
                                      color_mode='rgb',
                                      batch_size=32,
                                      class_mode='categorical',
                                      subset='training',
                                      shuffle=True)

In [ ]:
train_gen1.class_indices,train_gen2.class_indices

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)))
model.add(MaxPool2D())
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(64,(5,5),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit_generator(train_gen1,epochs=5)

In [ ]:
model.fit_generator(train_gen2,
                    epochs=5)

In [ ]:
train_gen3 = Datagen.flow_from_directory(
    directory=path_train3,
    target_size=(224,224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    subset='training',
    shuffle=True
)

In [ ]:
model.fit_generator(train_gen3,
                    epochs=5)

In [ ]:
score = model.evaluate_generator(train_gen2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
import cv2

In [ ]:
def get_prediction(path):
    A=cv2.imread(path)
    A=cv2.resize(A,(224,224))
    A=A.reshape(1,224,224,3)/255
    p=model.predict(A).argmax()
    return list(train_gen1.class_indices.keys())[p]

In [ ]:
folder_location ='../input/agriculture-crop-images/test_crop_image/'
file_names=os.listdir(folder_location)
c=0

for file in file_names:
    names = get_prediction(folder_location+file)
    print(file,"\t--->",names,'\n')
    if file[:3]==names[:3]:
        c = c+1 
print('Test_accuracy :',c/len(file_names))
